In [1]:
%run supervised_functions.ipynb

Calculating probability for start mol sampling


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 69304/69304 [01:27<00:00, 794.68it/s]


# Data

In [ ]:
def generate_train_data(smile, steps):
    mol = Chem.MolFromSmiles(smile)

    df = pd.DataFrame(columns=['reactant', 'rsub', 'rcen', 'rsig', 'rsig_cs_indices', 'psub', 'pcen', 'psig', 'psig_cs_indices', 'product', "step"])
    index = []
    
    # Get sequences
    try:
        for i in range(steps):
            actions = get_applicable_actions(mol)
            if actions.shape[0] == 0:
                raise Exception("No actions applicable.....")

            # Apply a random action
            rand_idx = np.random.randint(0, actions.shape[0])
            product = apply_action(mol, *actions.iloc[rand_idx])

            # Add it to df
            df.loc[df.shape[0], :] = [Chem.MolToSmiles(mol)] + actions.iloc[rand_idx].tolist() + [Chem.MolToSmiles(product), i]
            index.append(actions.iloc[rand_idx].name)

            # Next reactant = product
            mol = product

        # Fix index
        df.index = index

        # Fix target
        df["product"] = Chem.MolToSmiles(product)

        # Fix steps
        df["step"] = df.shape[0] - df["step"]
    except Exception as e:
        return pd.DataFrame(columns=['reactant', 'rsub', 'rcen', 'rsig', 'rsig_cs_indices', 'psub', 'pcen', 'psig', 'psig_cs_indices', 'product', 'step'])
    
        
    return df.sample(1)

main_df_dict = {}
num_trajectories_for_test = 10000
for steps in [1, 2, 5, 10]:
    N = steps * num_trajectories_for_test

    df_list = []
    final_shape = 0
    smiles_per_random_sample = 1000
    pool_chunk_size = 10

    # Create dataset for multi-step pred
    with Pool(30) as p, tqdm.tqdm(total=N) as pbar:
        while final_shape < N:
            smiles = np.random.choice(start_mols, size=(smiles_per_random_sample,), p=categorical_probs_for_sampling_start_mols)

            for new_df in p.imap_unordered(functools.partial(generate_train_data, steps=steps), smiles, chunksize=10):
                df_list.append(new_df)
                final_shape += new_df.shape[0]

            pbar.update(final_shape - pbar.n)

    main_df_dict[steps] = pd.concat(df_list)
    print(f"Steps = {steps}", main_df_dict[steps].shape)

import pickle
pickle.dump(main_df_dict, open("models/supervised/evaluation_dict.pickle", 'wb'))

In [2]:
import pickle
main_df_dict = pickle.load(open("models/supervised/evaluation_dict.pickle", 'rb'))

In [24]:
for i in main_df_dict:
    print(main_df_dict[i].shape)
#     main_df_dict[i] = main_df_dict[i].iloc[:1000]    

(1000, 11)
(1000, 11)
(1000, 11)
(1000, 11)


In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

### Action dataset

In [8]:
action_dataset = pd.read_csv("datasets/my_uspto/action_dataset-filtered.csv", index_col=0)
action_dataset = action_dataset.loc[action_dataset["action_tested"] & action_dataset["action_works"]]
action_dataset = action_dataset[["rsub", "rcen", "rsig", "rbond", "psub", "pcen", "psig", "pbond"]]
print(action_dataset.shape)

action_rsigs = data.Molecule.pack(list(map(molecule_from_smile, action_dataset["rsig"])))
action_psigs = data.Molecule.pack(list(map(molecule_from_smile, action_dataset["psig"])))

(89384, 8)


/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Li`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Ge`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `K`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Na`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Ti`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_

### Correct indices and applicable indices

In [25]:
# I'm storing as lists, so doing numpy operations for the elements
correct_applicable_indices = {steps: [] for steps in [1, 2, 5, 10]}
correct_action_dataset_indices = {steps: [] for steps in [1, 2, 5, 10]}
action_embedding_indices = {steps: [] for steps in [1, 2, 5, 10]}

# for indices_used_for_data, correct_idx in tqdm.tqdm(map(get_emb_indices_and_correct_idx, main_df_dict[steps].iterrows()), total=main_df_dict[steps].shape[0]):
for steps in [1, 2, 5, 10]:
    with Pool(20) as p:
        for indices_used_for_data, correct_app_idx, correct_act_idx in tqdm.tqdm(p.imap(get_emb_indices_and_correct_idx, main_df_dict[steps].iterrows(), chunksize=50), total=main_df_dict[steps].shape[0]):
            action_embedding_indices[steps].append(indices_used_for_data)
            correct_applicable_indices[steps].append(correct_app_idx)
            correct_action_dataset_indices[steps].append(correct_act_idx)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 214.44it/s]


### Test data

In [26]:
%%time
_device = device
device = 'cpu'
test_reactants = {steps: data.Molecule.pack(list(map(molecule_from_smile, main_df_dict[steps]["reactant"]))).to(device) for steps in [1, 2, 5, 10]}
test_products = {steps: data.Molecule.pack(list(map(molecule_from_smile, main_df_dict[steps]["product"]))).to(device) for steps in [1, 2, 5, 10]}
test_rsigs = {steps: data.Molecule.pack(list(map(molecule_from_smile, main_df_dict[steps]["rsig"]))).to(device) for steps in [1, 2, 5, 10]}
test_psigs = {steps: data.Molecule.pack(list(map(molecule_from_smile, main_df_dict[steps]["psig"]))).to(device) for steps in [1, 2, 5, 10]}

for steps in [1, 2, 5, 10]:
    print(steps, "--", test_reactants[steps].batch_size, test_products[steps].batch_size, test_rsigs[steps].batch_size, test_psigs[steps].batch_size)
device = _device

/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Li`
  warnings.warn("Unknown value `%s`" % x)
/home/abhor/miniconda3/envs/de_novo/lib/python3.7/site-packages/torchdrug-0.2.0-py3.7.egg/torchdrug/data/feature.py:42: UserWarning: Unknown value `Al`
  warnings.warn("Unknown value `%s`" % x)


1 -- 1000 1000 1000 1000
2 -- 1000 1000 1000 1000
5 -- 1000 1000 1000 1000
10 -- 1000 1000 1000 1000
CPU times: user 1min 18s, sys: 3.92 s, total: 1min 22s
Wall time: 24 s


### mse model

In [27]:
top_percentile = 90
batch_size = 128
metric_df_dict = {}

def construct_stats_from_rank_list(rank_list, total, dist, df=None):
    if df is None:
        df = pd.DataFrame(columns=[f"{distance_type}_{metric}" for distance_type in ["euc", "cos"] for metric in ["mean", "std", "total", "<10 %", "<5 %", "<1 %"]])
    
    argsort = np.argsort(rank_list)
    rl = np.array(rank_list)[argsort]
    total = np.array(total)[argsort]

    # print stats
    for step in range(1, steps+3): # steps + 3 because I start from 1 and do two extra for average and extrapolation at the end
        # Do a percentile to avoid outliers
        rl_perc = rl[:int(rl.shape[0] * top_percentile/100)]
        total_perc = total[:int(total.shape[0] * top_percentile/100)]
        
        if step == steps + 1: # do average
            rl_step = rl_perc
            total_step = total_perc
            step = "overall"
        elif step == steps + 2: # do extropolation to 1000
            factor = 1000/total_perc.mean()
            rl_step = rl_perc * factor
            total_step = total_perc * factor
            step = "extrapo"
        else:
            rl_step = rl_perc[(main_df_dict[steps]["step"] == step)[:rl_perc.shape[0]]]
            total_step = total_perc[(main_df_dict[steps]["step"] == step)[:rl_perc.shape[0]]]

        # Results
        df.loc[step, f"{dist[:3]}_mean"] = round(rl_step.mean(), 2)
        df.loc[step, f"{dist[:3]}_std"] = round(rl_step.std(), 2)
        df.loc[step, f"{dist[:3]}_total"] = round(np.mean(total_step), 2)
#             df.loc[step, f"{dist[:3]}_count"] = total_step.shape[0]
        df.loc[step, f"{dist[:3]}_<10 %"] = round((rl_step <= 10).sum() / rl_step.shape[0] * 100, 2)
        df.loc[step, f"{dist[:3]}_<5 %"] = round((rl_step <= 5).sum() / rl_step.shape[0] * 100, 2)
        df.loc[step, f"{dist[:3]}_<1 %"] = round((rl_step <= 1).sum() / rl_step.shape[0] * 100, 2)
    return df

In [28]:
import glob

model_name = "models/zinc2m_gin.pth"
gin_model = torch.load(model_name).to(device)

def mse_model_stats(actor, steps):
    pred = torch.concatenate([actor(torch.concatenate([get_mol_embedding(gin_model, test_reactants[steps][i:i+batch_size].to(device)), 
                                     get_mol_embedding(gin_model, test_products[steps][i:i+batch_size].to(device))], axis=1).detach()) for i in range(0, test_reactants[steps].batch_size-batch_size, batch_size)], axis=0)
    
    action_embeddings = get_action_dataset_embeddings(gin_model)

    # Rank list
    df = None
    for dist in ["euclidean", "cosine"]:
        rank_list = []
        total = []
        for i in range(pred.shape[0]):
            pred_for_i = pred[i]
            act_emb_for_i, correct_index = action_embeddings[action_embedding_indices[steps][i]], correct_applicable_indices[steps][i]

            rank, list_of_indices = get_ranking(pred_for_i, act_emb_for_i, correct_index, distance=dist)
            rank_list.append(rank.item())
            total.append(act_emb_for_i.shape[0])
            
        df = construct_stats_from_rank_list(rank_list, total, dist, df)
        
    return df

In [29]:
for steps in [1, 2, 5, 10]:
    path_string = f"models/supervised/mse_model/{min(steps, 5)}step.pth"
    assert len(glob.glob(path_string)) == 1
    actor = torch.load(glob.glob(path_string)[0]).to(device)
    print("STEPS =", steps)
    key = f"mse_model||steps={steps}"
    metric_df_dict[key] = mse_model_stats(actor, steps)
    display(metric_df_dict[key])
    print()
    print()

STEPS = 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:04<00:00,  9.93it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,12.11,20.5,1623.15,71.84,60.67,31.14,11.47,20.67,1607.71,75.81,65.14,33.5
overall,12.11,20.5,1623.15,71.84,60.67,31.14,11.47,20.67,1607.71,75.81,65.14,33.5
extrapo,7.46,12.63,1000.0,79.53,68.73,31.14,7.13,12.86,1000.0,83.0,71.59,33.5




STEPS = 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:04<00:00, 10.77it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:04<00:00, 11.00it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,137.11,191.07,513.55,34.55,25.45,10.3,125.35,179.57,603.38,35.15,27.27,12.73
2,101.77,154.57,482.97,33.12,25.48,9.55,92.85,145.05,482.24,36.94,26.11,12.74
3,108.74,157.85,447.81,39.01,30.22,11.54,99.13,147.45,435.47,40.11,34.07,15.93
4,92.3,157.85,476.52,37.17,19.47,6.19,85.42,149.69,470.59,39.82,20.35,7.96
5,84.75,157.47,426.86,48.15,34.39,12.17,77.69,147.87,402.39,51.85,36.51,15.34
overall,105.26,165.52,467.23,38.83,27.79,10.3,96.32,155.41,476.12,41.19,29.78,13.4
extrapo,225.29,354.26,1000.0,25.06,16.25,0.0,202.31,326.41,1000.0,26.8,18.73,0.0




STEPS = 10


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:04<00:00, 10.90it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,44.31,66.85,313.49,39.58,22.92,4.17,41.75,62.53,263.19,39.58,21.88,9.38
2,74.7,99.24,220.0,36.0,22.0,8.0,69.24,90.68,314.04,34.0,22.0,8.0
3,54.02,101.05,196.86,50.0,34.0,9.0,50.51,93.49,258.58,50.0,34.0,10.0
4,62.63,103.93,216.34,51.22,36.59,14.63,59.78,99.39,229.71,48.78,36.59,20.73
5,57.35,86.14,223.6,42.68,25.61,12.2,53.46,79.96,164.18,41.46,24.39,13.41
6,56.72,97.67,201.08,50.6,37.35,9.64,52.78,88.71,186.54,49.4,37.35,14.46
7,61.25,94.54,221.82,33.33,19.54,3.45,57.95,89.77,251.31,28.74,19.54,3.45
8,27.01,73.12,160.13,74.67,66.67,36.0,25.56,69.08,177.48,74.67,66.67,37.33
9,97.36,124.28,373.44,31.58,22.11,4.21,91.05,114.51,380.11,29.47,22.11,7.37
10,54.46,77.85,193.57,39.29,30.36,7.14,50.96,72.55,219.18,37.5,30.36,12.5


### Actor only

In [30]:
import glob

batch_size = 128
def actor_only_stats(actor, steps):
    pred = torch.concatenate([actor(test_reactants[steps][i:i+batch_size].to(device), 
                                     test_products[steps][i:i+batch_size].to(device)).detach() for i in range(0, test_reactants[steps].batch_size-batch_size, batch_size)], axis=0)

    action_embeddings = get_action_dataset_embeddings(actor.GIN)

    # Rank list
    df = None
    for dist in ["euclidean", "cosine"]:
        rank_list = []
        total = []
        for i in range(pred.shape[0]):
            pred_for_i = pred[i]
            act_emb_for_i, correct_index = action_embeddings[action_embedding_indices[steps][i]], correct_applicable_indices[steps][i]

            rank, list_of_indices = get_ranking(pred_for_i, act_emb_for_i, correct_index, distance=dist)
            rank_list.append(rank.item())
            total.append(act_emb_for_i.shape[0])
        
        df = construct_stats_from_rank_list(rank_list, total, dist, df)
        
    return df
#         print("\t".join(list(map(str, df.values[0]))))

In [31]:
for steps in [1, 2, 5, 10]:
    path_string = f"models/supervised/actor/*steps={min(steps, 5)}*/model.pth"
#     path_string = f"models/supervised/actor/1step/*/model.pth"
    assert len(glob.glob(path_string)) == 1
    actor = torch.load(glob.glob(path_string)[0]).to(device)
    print("STEPS =", steps)
    key = f"actor_only||steps={steps}"
    metric_df_dict[key] = actor_only_stats(actor, steps)
    display(metric_df_dict[key])
    print()
    print()

STEPS = 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:04<00:00, 10.04it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,1.64,1.24,1649.96,100.0,97.27,69.98,1.32,0.63,1668.32,100.0,100.0,76.43
overall,1.64,1.24,1649.96,100.0,97.27,69.98,1.32,0.63,1668.32,100.0,100.0,76.43
extrapo,0.99,0.75,1000.0,100.0,100.0,69.98,0.79,0.38,1000.0,100.0,100.0,76.43




STEPS = 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:04<00:00,  9.98it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:04<00:00, 10.42it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,121.8,190.32,540.56,38.18,32.73,16.36,99.43,162.13,579.27,41.21,35.15,20.0
2,85.82,148.75,460.52,41.4,29.94,16.56,69.62,127.08,448.43,45.86,33.76,20.38
3,91.98,150.81,484.31,41.76,37.36,20.33,73.59,125.51,465.41,45.6,39.01,23.63
4,80.37,155.36,387.29,45.13,30.97,11.5,65.92,132.17,514.88,51.33,38.94,15.04
5,74.16,153.43,402.56,56.08,43.92,19.58,60.72,130.2,416.79,60.32,49.74,24.87
overall,91.08,161.39,458.42,44.79,35.61,17.37,74.01,136.73,480.95,49.01,39.7,21.34
extrapo,198.68,352.06,1000.0,32.01,24.81,0.0,153.89,284.29,1000.0,37.1,28.29,0.0




STEPS = 10


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:04<00:00, 10.04it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,42.34,63.21,214.81,41.67,28.12,10.42,35.8,55.76,263.33,44.79,31.25,11.46
2,70.82,91.33,292.82,36.0,24.0,8.0,61.36,81.33,215.5,40.0,32.0,10.0
3,49.8,90.8,233.42,52.0,42.0,10.0,43.35,81.25,341.16,55.0,44.0,16.0
4,59.33,96.84,243.5,52.44,39.02,21.95,51.8,86.6,199.52,57.32,43.9,24.39
5,54.15,79.38,244.55,42.68,29.27,13.41,46.76,70.51,240.63,47.56,34.15,14.63
6,52.6,86.53,192.2,50.6,40.96,15.66,45.93,77.25,203.23,56.63,44.58,18.07
7,58.07,87.52,199.95,34.48,20.69,3.45,49.99,78.24,254.97,42.53,24.14,6.9
8,24.84,66.08,137.75,74.67,66.67,40.0,21.97,60.11,124.52,76.0,73.33,49.33
9,91.22,111.31,341.32,32.63,23.16,7.37,79.84,99.81,379.61,37.89,27.37,7.37
10,51.64,73.64,354.7,39.29,32.14,12.5,44.48,65.64,238.3,44.64,33.93,12.5


### Actor + critic

In [32]:
import glob

batch_size = 128
def actor_critic_separate_stats(actor, critic, k, steps):
    pred = torch.concatenate([actor(test_reactants[steps][i:i+batch_size].to(device), 
                                     test_products[steps][i:i+batch_size].to(device)).detach() for i in range(0, test_reactants[steps].batch_size-batch_size, batch_size)], axis=0)

    action_embeddings = get_action_dataset_embeddings(actor.GIN)

    # Rank list
    df = None
    for dist in ["euclidean", "cosine"]:
        rank_list = []
        total = []
        dict_of_list_of_indices = {}

        for i in tqdm.tqdm(range(pred.shape[0])):
            pred_for_i = pred[i]
            act_emb_for_i, correct_index = action_embeddings[action_embedding_indices[steps][i]], correct_applicable_indices[steps][i]

            # Get default rank
            rank, list_of_indices = get_ranking(pred_for_i, act_emb_for_i, correct_index, distance=dist)
            rank = rank.item()
            rank_list.append(rank)
            total.append(act_emb_for_i.shape[0])

            # Save list of indices for critic
            list_of_indices = get_top_k_indices(pred_for_i, act_emb_for_i, correct_index, distance=dist, k=k).detach().cpu().numpy()
            if correct_index in list_of_indices:
                dict_of_list_of_indices[i] = list_of_indices

        # Post process with critic
        i_sorted = sorted(list(dict_of_list_of_indices.keys()))
        action_indices = np.concatenate([action_embedding_indices[steps][i][dict_of_list_of_indices[i]] for i in i_sorted])
        state_indices = np.concatenate([np.full_like(dict_of_list_of_indices[i], i) for i in i_sorted])
        critic_batch = 1024
        critic_qs = []
        for i in tqdm.tqdm(range(0, action_indices.shape[0], critic_batch)):
            batch_reactants = test_reactants[steps][state_indices[i:i+critic_batch]]
            batch_products = test_products[steps][state_indices[i:i+critic_batch]]
            batch_rsigs = action_rsigs[action_indices[i:i+critic_batch]]
            batch_psigs = action_psigs[action_indices[i:i+critic_batch]]
            critic_qs.append(critic(batch_reactants.to(device), batch_products.to(device), batch_rsigs.to(device), batch_psigs.to(device)).detach().cpu().numpy())

        critic_qs = np.concatenate(critic_qs)

        start = 0
        for i in i_sorted:
            end = start + dict_of_list_of_indices[i].shape[0]
            i_critic_qs = critic_qs[start:end]
            rank = (dict_of_list_of_indices[i][i_critic_qs.reshape(-1).argsort()[::-1]] == correct_index).argmax() + 1
            rank_list[i] = rank
            start = end

        df = construct_stats_from_rank_list(rank_list, total, dist, df)
    return df

In [33]:
for k in [10, 20, 50]:
    for steps in [1, 2, 5, 10]:
        path_string = f"models/supervised/actor/*steps={min(steps, 5)}*/model.pth"
        assert len(glob.glob(path_string)) == 1
        actor = torch.load(glob.glob(path_string)[0]).to(device)

        path_string = f"models/supervised/critic/*steps={min(steps, 5)}*/model.pth"
        assert len(glob.glob(path_string)) == 1
        critic = torch.load(glob.glob(path_string)[0]).to(device)

        print("STEPS =", steps, " || K = ", k)
        key = f"actor+critic||steps={steps}||criticK={k}"
        metric_df_dict[key] = actor_critic_separate_stats(actor, critic, k, steps)
        display(metric_df_dict[key])
        print()
        print()

STEPS = 1  || K =  10


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00,  9.39it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,1.0,0.0,1647.05,100.0,100.0,100.0,1.0,0.0,1680.06,100.0,100.0,100.0
overall,1.0,0.0,1647.05,100.0,100.0,100.0,1.0,0.0,1680.06,100.0,100.0,100.0
extrapo,0.61,0.0,1000.0,100.0,100.0,100.0,0.6,0.0,1000.0,100.0,100.0,100.0




STEPS = 2  || K =  10


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00,  7.65it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:02<00:00,  6.63it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,120.07,191.32,560.15,44.24,43.03,42.42,97.76,163.05,575.54,46.06,45.45,43.64
2,83.43,149.97,467.88,49.68,48.41,46.5,67.27,128.17,439.36,54.14,52.87,49.68
3,90.35,151.71,415.34,48.35,46.7,46.15,71.92,126.37,477.86,52.2,48.9,46.7
4,76.9,156.93,454.8,61.06,57.52,52.21,62.73,133.53,490.17,62.83,61.06,61.06
5,71.66,154.51,405.4,64.02,61.9,61.38,58.47,131.1,430.37,68.25,65.61,64.02
overall,88.82,162.53,458.42,53.23,51.36,49.88,71.86,137.75,480.95,56.58,54.59,52.73
extrapo,193.74,354.55,1000.0,50.87,50.12,0.0,149.42,286.41,1000.0,54.22,52.98,0.0




STEPS = 10  || K =  50


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:03<00:00,  5.33it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,39.41,64.5,248.22,44.79,43.75,41.67,32.84,56.8,188.53,52.08,44.79,42.71
2,68.26,92.88,342.04,42.0,38.0,36.0,59.14,82.59,332.26,44.0,40.0,38.0
3,47.31,91.81,271.01,56.0,54.0,51.0,40.91,82.12,214.21,59.0,56.0,54.0
4,57.13,97.93,251.61,58.54,54.88,52.44,49.67,87.56,228.3,62.2,58.54,54.88
5,51.62,80.66,219.79,51.22,45.12,42.68,44.23,71.69,285.87,52.44,47.56,45.12
6,50.42,87.59,221.0,59.04,54.22,50.6,43.92,78.18,296.78,61.45,56.63,54.22
7,54.83,89.14,227.09,43.68,39.08,33.33,46.89,79.63,179.17,47.13,42.53,37.93
8,23.32,66.37,150.99,77.33,74.67,74.67,20.79,60.31,269.87,77.33,76.0,74.67
9,89.12,112.8,312.79,40.0,35.79,32.63,78.02,101.06,315.33,40.0,38.95,35.79
10,49.32,74.82,156.55,46.43,41.07,39.29,42.14,66.66,274.09,51.79,46.43,41.07


### Actor(mse)-critic

In [34]:
import glob

batch_size = 128
def actor_critic_stats(ac, k, steps):
    pred = torch.concatenate([ac(test_reactants[steps][i:i+batch_size].to(device), 
                                     test_products[steps][i:i+batch_size].to(device), None, None, "actor").detach() for i in range(0, test_reactants[steps].batch_size-batch_size, batch_size)], axis=0)

    action_embeddings = get_action_dataset_embeddings(ac.GIN)

    # Rank list
    df = None
    for dist in ["euclidean", "cosine"]:
        rank_list = []
        total = []
        dict_of_list_of_indices = {}

        for i in tqdm.tqdm(range(pred.shape[0])):
            pred_for_i = pred[i]
            act_emb_for_i, correct_index = action_embeddings[action_embedding_indices[steps][i]], correct_applicable_indices[steps][i]

            # Get default rank
            rank, list_of_indices = get_ranking(pred_for_i, act_emb_for_i, correct_index, distance=dist)
            rank = rank.item()
            rank_list.append(rank)
            total.append(act_emb_for_i.shape[0])

            # Save list of indices for critic
            list_of_indices = get_top_k_indices(pred_for_i, act_emb_for_i, correct_index, distance=dist, k=k).detach().cpu().numpy()
            if correct_index in list_of_indices:
                dict_of_list_of_indices[i] = list_of_indices

        # Post process with critic
        i_sorted = sorted(list(dict_of_list_of_indices.keys()))
        action_indices = np.concatenate([action_embedding_indices[steps][i][dict_of_list_of_indices[i]] for i in i_sorted])
        state_indices = np.concatenate([np.full_like(dict_of_list_of_indices[i], i) for i in i_sorted])
        critic_batch = 1024
        critic_qs = []
        for i in tqdm.tqdm(range(0, action_indices.shape[0], critic_batch)):
            batch_reactants = test_reactants[steps][state_indices[i:i+critic_batch]]
            batch_products = test_products[steps][state_indices[i:i+critic_batch]]
            batch_rsigs = action_rsigs[action_indices[i:i+critic_batch]]
            batch_psigs = action_psigs[action_indices[i:i+critic_batch]]
            critic_qs.append(ac(batch_reactants.to(device), batch_products.to(device), batch_rsigs.to(device), batch_psigs.to(device), "critic").detach().cpu().numpy())

        critic_qs = np.concatenate(critic_qs)

        start = 0
        for i in tqdm.tqdm(i_sorted):
            end = start + dict_of_list_of_indices[i].shape[0]
            i_critic_qs = critic_qs[start:end]
            rank = (dict_of_list_of_indices[i][i_critic_qs.reshape(-1).argsort()[::-1]] == correct_index).argmax() + 1
            rank_list[i] = rank
            start = end

            df = construct_stats_from_rank_list(rank_list, total, dist, df)
    return df

In [35]:
# SUpervised
for actor_loss in ["mse", "triplet"]:
    for k in [10, 20, 50]:
        for steps in [1, 2, 5, 10]:
            path_string = f"models/supervised/actor-critic/*{actor_loss}*steps={min(steps, 5)}*/model.pth"
            assert len(glob.glob(path_string)) == 1
            ac = torch.load(glob.glob(path_string)[0]).to(device)

            print(f"ACTOR LOSS = {actor_loss} || K = {k} || STEPS = {steps}")
            key = f"actor-critic({actor_loss})||steps={steps}||criticK={k}"
            metric_df_dict[key] = actor_critic_stats(ac, k, steps)
            display(metric_df_dict[key])
            print()
            print()

ACTOR LOSS = mse || K = 10 || STEPS = 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 881/881 [00:02<00:00, 380.48it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,1.0,0.0,1675.58,100.0,100.0,100.0,1.0,0.0,1682.48,100.0,100.0,100.0
overall,1.0,0.0,1675.58,100.0,100.0,100.0,1.0,0.0,1682.48,100.0,100.0,100.0
extrapo,0.6,0.0,1000.0,100.0,100.0,100.0,0.59,0.0,1000.0,100.0,100.0,100.0




ACTOR LOSS = mse || K = 10 || STEPS = 2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 577/577 [00:02<00:00, 263.33it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 610/610 [00:04<00:00, 142.18it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,84.59,145.66,607.48,49.7,47.88,46.67,77.59,134.53,543.73,51.52,49.09,47.88
2,56.89,112.05,490.29,56.69,55.41,54.14,52.47,104.74,471.04,58.6,56.05,55.41
3,61.18,110.91,459.14,54.95,53.3,52.2,56.49,103.28,493.91,54.95,53.85,52.75
4,51.85,112.39,381.16,65.49,63.72,62.83,48.89,106.9,543.2,67.26,64.6,63.72
5,49.92,114.77,424.72,70.37,69.31,68.78,45.98,106.27,389.84,70.37,69.84,69.31
overall,61.19,120.76,476.57,59.31,57.82,56.82,56.5,112.38,482.16,60.3,58.56,57.69
extrapo,128.39,253.4,1000.0,57.57,57.07,0.0,117.18,233.08,1000.0,58.56,57.82,0.0




ACTOR LOSS = triplet || K = 50 || STEPS = 10


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 637/637 [00:08<00:00, 77.26it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,29.24,51.66,199.48,52.08,44.79,43.75,25.36,46.63,212.78,58.33,52.08,44.79
2,52.46,74.59,214.46,44.0,42.0,40.0,46.66,66.96,301.1,46.0,42.0,42.0
3,36.46,74.77,244.06,61.0,56.0,54.0,32.27,67.71,222.35,64.0,58.0,56.0
4,44.35,78.76,250.09,63.41,58.54,57.32,40.12,72.06,234.48,64.63,60.98,58.54
5,39.83,65.61,195.44,54.88,51.22,46.34,34.9,58.78,279.93,56.1,52.44,51.22
6,39.83,73.36,240.11,62.65,59.04,56.63,34.81,64.9,234.12,63.86,61.45,57.83
7,41.29,71.23,347.54,50.57,43.68,41.38,36.8,65.62,188.4,52.87,47.13,43.68
8,18.51,54.34,217.49,80.0,77.33,76.0,16.37,49.0,322.49,82.67,77.33,77.33
9,69.57,92.38,379.77,43.16,40.0,37.89,62.06,83.42,398.14,45.26,40.0,40.0
10,37.34,60.09,270.23,55.36,46.43,41.07,32.27,53.42,336.34,57.14,50.0,46.43


In [36]:
for k in [10, 20, 50]:
    for steps in [1, 2, 5, 10]:
        path_string = f"models/supervised/offlineRL/*steps={min(steps, 5)}*/model.pth"
        assert len(glob.glob(path_string)) == 1
        ac = torch.load(glob.glob(path_string)[0]).to(device)

        print(f"K = {k} || STEPS = {steps}")
        key = f"actor-critic(PG)||steps={steps}||criticK={k}"
        metric_df_dict[key] = actor_critic_stats(ac, k, steps)
        display(metric_df_dict[key])
        print()
        print()

K = 10 || STEPS = 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 888/888 [00:02<00:00, 384.91it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,1.0,0.0,1684.68,100.0,100.0,100.0,1.0,0.0,1679.77,100.0,100.0,100.0
overall,1.0,0.0,1684.68,100.0,100.0,100.0,1.0,0.0,1679.77,100.0,100.0,100.0
extrapo,0.59,0.0,1000.0,100.0,100.0,100.0,0.6,0.0,1000.0,100.0,100.0,100.0




K = 10 || STEPS = 2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 615/615 [00:02<00:00, 265.47it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 580/580 [00:04<00:00, 143.38it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,98.96,157.44,545.02,47.88,45.45,45.45,95.57,156.1,521.71,47.88,45.45,44.85
2,69.45,125.26,440.38,55.41,53.5,50.96,66.35,123.12,529.73,55.41,52.87,49.68
3,74.79,126.2,472.57,52.75,51.1,48.9,71.67,124.61,506.65,52.75,51.1,48.9
4,61.75,125.99,539.65,63.72,61.95,61.06,60.73,127.07,441.12,63.72,61.06,61.06
5,58.85,127.49,472.89,69.31,67.2,65.61,57.21,127.02,425.59,69.31,66.67,64.55
overall,73.13,134.05,490.61,57.69,55.71,54.22,70.6,132.97,486.03,57.69,55.33,53.6
extrapo,149.06,273.23,1000.0,55.33,54.71,0.0,145.26,273.59,1000.0,54.71,54.34,0.0




K = 50 || STEPS = 10


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 614/614 [00:07<00:00, 78.45it/s]


,euc_mean,euc_std,euc_total,euc_<10 %,euc_<5 %,euc_<1 %,cos_mean,cos_std,cos_total,cos_<10 %,cos_<5 %,cos_<1 %
1,34.21,57.86,196.04,51.04,44.79,42.71,30.02,52.34,169.45,52.08,44.79,43.75
2,59.84,83.24,256.5,42.0,40.0,38.0,53.3,75.21,300.68,44.0,42.0,38.0
3,41.24,82.15,260.59,56.0,55.0,53.0,37.26,76.06,206.56,61.0,56.0,54.0
4,49.63,86.1,290.01,60.98,57.32,53.66,44.65,78.92,330.73,63.41,58.54,56.1
5,45.04,71.52,325.29,52.44,47.56,42.68,40.13,65.64,309.13,53.66,51.22,46.34
6,44.67,80.03,248.42,61.45,56.63,50.6,39.88,73.71,332.1,62.65,59.04,55.42
7,47.33,78.32,200.93,44.83,41.38,35.63,42.17,72.01,205.72,50.57,43.68,40.23
8,20.93,60.28,246.27,77.33,76.0,74.67,18.85,55.49,190.45,78.67,77.33,74.67
9,78.57,101.63,343.47,40.0,37.89,33.68,71.32,94.53,328.85,41.05,40.0,35.79
10,42.64,66.39,202.7,48.21,41.07,41.07,37.82,59.61,287.55,53.57,46.43,41.07


In [37]:
metric_df_dict.keys()

dict_keys(['mse_model||steps=1', 'mse_model||steps=2', 'mse_model||steps=5', 'mse_model||steps=10', 'actor_only||steps=1', 'actor_only||steps=2', 'actor_only||steps=5', 'actor_only||steps=10', 'actor+critic||steps=1||criticK=10', 'actor+critic||steps=2||criticK=10', 'actor+critic||steps=5||criticK=10', 'actor+critic||steps=10||criticK=10', 'actor+critic||steps=1||criticK=20', 'actor+critic||steps=2||criticK=20', 'actor+critic||steps=5||criticK=20', 'actor+critic||steps=10||criticK=20', 'actor+critic||steps=1||criticK=50', 'actor+critic||steps=2||criticK=50', 'actor+critic||steps=5||criticK=50', 'actor+critic||steps=10||criticK=50', 'actor-critic(mse)||steps=1||criticK=10', 'actor-critic(mse)||steps=2||criticK=10', 'actor-critic(mse)||steps=5||criticK=10', 'actor-critic(mse)||steps=10||criticK=10', 'actor-critic(mse)||steps=1||criticK=20', 'actor-critic(mse)||steps=2||criticK=20', 'actor-critic(mse)||steps=5||criticK=20', 'actor-critic(mse)||steps=10||criticK=20', 'actor-critic(mse)||st

In [38]:
model_types = ["mse_model", "actor_only", "actor+critic", "actor-critic(mse)", "actor-critic(triplet)", "actor-critic(PG)"]
step_list = [1, 2, 5, 10]
critick_list = [10, 20, 50]

key_list = []
for s in step_list:
    for m in model_types:
        if "critic" not in m:
            key_list.append(f"{m}||steps={s}")
        else:
            for k in critick_list:
                key_list.append(f"{m}||steps={s}||criticK={k}")

df_index = "overall"
ultra_metric_df = []

for key in key_list:
    print("\t".join(list(map(str, metric_df_dict[key].loc[df_index].values))))

# for key in key_list:
#     series = metric_df_dict[key].loc[df_index]
#     series.name = key
#     ultra_metric_df.append(series)

# ultra_metric_df = pd.DataFrame(ultra_metric_df)

# for i in range(ultra_metric_df.shape[0]):
#     print("\t".join(list(map(str, ultra_metric_df.iloc[i].values))))

12.11	20.5	1623.15	71.84	60.67	31.14	11.47	20.67	1607.71	75.81	65.14	33.5
1.64	1.24	1649.96	100.0	97.27	69.98	1.32	0.63	1668.32	100.0	100.0	76.43
1.0	0.0	1647.05	100.0	100.0	100.0	1.0	0.0	1680.06	100.0	100.0	100.0
1.0	0.0	1666.26	100.0	100.0	100.0	1.0	0.0	1679.44	100.0	100.0	100.0
1.0	0.0	1654.48	100.0	100.0	100.0	1.0	0.0	1671.81	100.0	100.0	100.0
1.0	0.0	1675.58	100.0	100.0	100.0	1.0	0.0	1682.48	100.0	100.0	100.0
1.0	0.0	1680.77	100.0	100.0	100.0	1.0	0.0	1671.66	100.0	100.0	100.0
1.0	0.0	1673.75	100.0	100.0	100.0	1.0	0.0	1665.57	100.0	100.0	100.0
1.0	0.0	1679.14	100.0	100.0	100.0	1.0	0.0	1680.48	100.0	100.0	100.0
1.0	0.0	1675.02	100.0	100.0	100.0	1.0	0.0	1688.97	100.0	100.0	100.0
1.0	0.0	1674.33	100.0	100.0	100.0	1.0	0.0	1676.67	100.0	100.0	100.0
1.0	0.0	1684.68	100.0	100.0	100.0	1.0	0.0	1679.77	100.0	100.0	100.0
1.0	0.0	1671.26	100.0	100.0	100.0	1.0	0.0	1667.65	100.0	100.0	100.0
1.0	0.0	1659.55	100.0	100.0	100.0	1.0	0.0	1664.85	100.0	100.0	100.0
103.22	175.37	1009.56	44.04	36.1	18.11